In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

showHeader('Colorado disclosures',line2=f'Summary of XXXXXXX',use_remote=True)

In [ ]:
last_repo = datetime.datetime(year=2025,month=3,day=24)
law_begins = datetime.datetime(year=2023,month=7,day=31)

# fetch data set
df = pd.read_parquet(os.path.join(hndl.sandbox_dir,'workdf.parquet'),
               columns=['APINumber','DisclosureId','bgStateName','date','OperatorName','bgCAS','mass',
                       'bgCountyName','in_std_filtered','JobStartDate','TradeName'],
               filters=[('bgStateName','==','colorado'),
                        ('in_std_filtered','==',True),
                        ]
              )

df = df[df.date>law_begins]
df['start_date'] = pd.to_datetime(df.JobStartDate,errors='coerce').dt.date
df['start_date'] = pd.to_datetime(df.start_date,errors='coerce')
gb = df.groupby('DisclosureId',as_index=False)[['APINumber','OperatorName','bgCountyName',
                                                'start_date','date']].first()
# gb1 = gb.groupby('bgStateName',as_index=False)['DisclosureId'].count().rename({'DisclosureId':'Number of disclosures'},axis=1)

# gb2 = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count()
# gb2 = gb2.groupby('bgStateName',as_index=False)['bgCountyName'].count().rename({'bgCountyName':'Number of counties'},axis=1)

# gb3 = gb.groupby('bgStateName',as_index=False)['date'].agg(['min','max']).rename({'min':'earliest',
#                                                                                   'max':'latest'},axis=1)
# mg = pd.merge(gb1,gb2,on='bgStateName',how='left')
# mg = pd.merge(mg,gb3,on='bgStateName',how='left')
# mg = mg.rename({'bgStateName':'State'},axis=1)
# # mg = mg.reset_index(drop=True)
# mg.State = '<center><h3>'+mg.State.str.title().map(lambda x: th.getStateLink(x,x))+'</h3></center>'
# mg = mg.sort_values('Number of disclosures',ascending=False)

import itables.options as opt
opt.order = []  # no sorting

iShow(gb.reset_index(drop=True))
# gb.info()

work_dir = r"G:\My Drive\webshare\scrape_data\Colorado\ECMC_tracker"
welldf_fn = os.path.join(work_dir,'ECMC_welldf.parquet')

# First time only
# welldf = pd.DataFrame({'APINumber':[],'DisclosureId':[],
#                        'locationID':[],'OperatorName':[],
#                        'ECMC_op':[],'bgCountyName':[],
#                        'start_date':[],'date':[]})
# welldf.to_parquet(welldf_fn)
welldf = pd.read_parquet(welldf_fn)
welldf

### Populate welldf with current disclosures

In [ ]:
dlist = welldf.DisclosureId.unique().tolist()
c = ~gb.DisclosureId.isin(dlist)
both = pd.concat([welldf,gb[c]])
both

### Fetch COGIS data on all disclosures

In [ ]:
####  REDO WHEN ADDING NEW DATA

# import scrape.Colorado.CO_scraper_support as coss
# def make_soup_for_well(APINumber):
#     rooturl = "https://ecmc.state.co.us/cogisdb/Facility/FacilityDetailExpand?api="
#     url = rooturl+ APINumber[2:-4]
#     # print(url)
#     html_content = coss.fetch_html(url)
#     # print(html_content)
#     # print('\n'+url)
#     # print("Recycled water" in html_content)
#     return coss.make_soup(html_content)

# def get_location_id(soup):
#     # Find the <td> element containing "LocationID:"
#     location_id_td = soup.find('td', string='LocationID:')
    
#     if location_id_td:
#         # Get the next sibling (the <td> with the value)
#         value_td = location_id_td.find_next_sibling('td')
#         if value_td:
#             # Extract the text and convert it to an integer
#             location_id_value = int(value_td.text)
#             return location_id_value  # Output: 455576
#         else:
#             print("Error: Could not find sibling td with the value.")
#     else:
#         print("Error: Could not find td containing 'LocationID:'") 
    

# def get_ECMC_operator(soup):
#     # Find the <td> element containing "Operator:"
#     operator_td = soup.find('td', string='Operator:')
    
#     if operator_td:
#         # Get the next sibling (the <td> with the value)
#         value_td = operator_td.find_next_sibling('td')
#         if value_td:
#             # Extract the text
#             operator_value = value_td.text.strip()  # Remove leading/trailing spaces
#             output = operator_value.split('(')[0]
#             return output
#         else:
#             print("Error: Could not find sibling td with the value.")
#     else:
#         print("Error: Could not find td containing 'Operator:'")    

# #  fetch ECMC data for the new disclosures
# def get_ECMC_data(APINumber):
#     soup = make_soup_for_well(APINumber)
#     locID = get_location_id(soup)
#     ECMCop = get_ECMC_operator(soup)
#     return (locID,ECMCop)
# locs = []
# ops = []

# # tmp = both[:5].copy()
# for i,row in both.iterrows():
#     print(i,end=' ')
#     if type(row.locationID) == 'str':
#         print(f'skipping {row.APINumber}')
#         locs.append(row.locationID)
#         ops.append(row.ECMC_op)
#         continue
#     else:
#         tup = get_ECMC_data(row.APINumber)
#         locs.append(tup[0])
#         ops.append(tup[1])
# # 
# both.locationID = locs
# both.ECMC_op = ops
# both.to_parquet(welldf_fn)
# both

## get ECMC locations

In [ ]:
url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
t = pd.read_html(url)
# print(len(t))
eLocs = t[0].copy()
eLocs['locationID'] = eLocs['Location Number']
mg = pd.merge(both,eLocs[['locationID','Location Name','Last Disclosure Update']],on='locationID', how='outer')
print(f'Total number of unique "locations" (well pads): {len(mg.locationID.unique())}')
c = mg['Location Name'].notna()
print(f'Number of locations with an ECMC disclosure: {len(mg[c].locationID.unique())}')
mg['status'] = np.where(mg.APINumber.isna(),'no_FF','has_FF')
mg.status = np.where(mg['Location Name'].isna(), mg.status + ' no_ECMC', mg.status + ' has_ECMC')
mg

### Show data from one location

In [ ]:
target_loc = 477204
ff_api = mg[mg.locationID==target_loc].APINumber.unique().tolist()
print(f'Number of wells in FF at this location: {len(ff_api)}')
print(ff_api)
ff_cas = df[df.APINumber.isin(ff_api)].groupby('bgCAS', as_index=False)['mass'].sum()
ff_cas
ff_tn = df[df.APINumber.isin(ff_api)].groupby('TradeName', as_index=False).size()
ff_tn

In [ ]:
out = pd.read_html(f"https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Location/{target_loc}")
ecmc_cas = out[0]
casmg = pd.merge(ecmc_cas,ff_cas,left_on='CAS Number', right_on='bgCAS',how='outer')
casmg

In [ ]:
ecmc_form46a = out[1]
ecmc_form46a['date'] = pd.to_datetime(ecmc_form46a['Submit Date'])
c = ecmc_form46a.date==ecmc_form46a.date.max()
pdf_dir = r"G:\My Drive\webshare\scrape_data\Colorado\imaged_docs"
docnum = ecmc_form46a[c]['Document #'].tolist()[0]             
pdffn = os.path.join(pdf_dir,f'{docnum}.pdf')
pdffn

In [ ]:
known_ecmc_prod_names = pd.read_parquet(os.path.join(pdf_dir,'products_set.parquet'))
known_ecmc_prod_names

In [ ]:
import scrape.Colorado.CO_imaged_docs_extract as cide
text = cide.get_pdf_text(pdffn)
ecmc_prod_names = cide.extract_product_names(text)
ecmc_api10 = cide.extract_APINumbers(text)
print(ecmc_prod_names)

In [ ]:
prod_names = pd.DataFrame({'TradeName':df.TradeName.unique().tolist()})
prod_names.to_parquet(os.path.join(r"C:\MyDocs\integrated\gwa_local\tmp",'CO_prod_names.parquet'))

## get well data for spud date

In [ ]:
fn = r"G:\My Drive\webshare\scrape_data\Colorado\Wells.csv"
cowells = pd.read_csv(fn,low_memory=False,usecols=['API,C,10','Spud_Date,D','Operator,C,50'],
                     dtype={'API,C,10':'str'})
cowells.columns = ['api','operator','spud']
cowells['spud_date'] = pd.to_datetime(cowells.spud)
cowells['APINumber'] = '05'+cowells.api+'0000'
cowells = cowells.drop(['api','spud'],axis=1)
cowells

In [ ]:
mg = mg.merge(cowells[['spud_date','APINumber']],on='APINumber',how='left')
mg['spud_start_diff_days'] = (mg.start_date - mg.spud_date).dt.days
mg